todos
----
- dummy encode race and attraction and rerun classifiers
- binarize labels in general: cm,af.. ->1,2,3,4 

- try RF for this cell

```
X = np.c_[encodings[:-num_of_friends], df['rating'][:-num_of_friends]]

rf = LogisticRegression(class_weight='balanced').fit(X, colors_dic[:-num_of_friends])
get_multi_label_roc_score(pd.Series(colors_dic[:-num_of_friends]), encodings[:-num_of_friends] ,rf)
```

- why didn't the LR pick-up the labels in the feature matrix?


In [1]:
from image_helpers import *
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')


/Users/ajay/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
!subl image_helpers.py

In [3]:
_, paths, encodings, arrays_rescaled, X = load_encodings('../data/CM_ALL_faces_encodings.pickle')
cm_end_idx = len(paths)
paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/AM_ALL_faces_encodings.pickle')
am_end_idx = len(paths)
paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/AF_ALL_faces_encodings.pickle')
af_end_idx = len(paths)
paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/CF_ALL_faces_encodings.pickle')
cf_end_idx = len(paths)

paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/friends_and_myself_pics_faces_encodings.pickle')
female_friends_end_idx = len(paths)
paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/friends_pics_male_faces_encodings.pickle')
male_friends_end_idx = len(paths)

2739 2739 2739 2739
4359 4359 4359 4359
5100 5100 5100 5100
5116 5116 5116 5116
5124 5124 5124 5124


In [4]:
df = pd.DataFrame().from_csv(
        '../data/filename_to_rating_lookup_ALL.csv', 
        index_col=[0]
    ) 
    
look_up_file_to_attractiveness_rating = dict(zip(df['Filename'], df['Rating']))
paths = [file_path.replace('_face', '').split('/')[-1] for file_path in paths]
colors_dic = [look_up_file_to_attractiveness_rating.get(just_file_name) for just_file_name in paths]
pd.Series(colors_dic).value_counts()


attractiveness_colors_lis=[]
for color_number in colors_dic:
    if not color_number:
        #friends are blue
        attractiveness_colors_lis.append(-1)
    else:
        attractiveness_colors_lis.append(color_number)

pd.Series(attractiveness_colors_lis).value_counts()

 3    2415
 4    1056
 2     738
 5     731
 1     160
-1      24
dtype: int64

In [5]:
num_of_friends = 24

labels = ['cm' for _ in range(cm_end_idx)] + \
['am' for _ in range(am_end_idx - cm_end_idx)] + \
['af' for _ in range(af_end_idx - am_end_idx )] + \
['cf' for _ in range(cf_end_idx - af_end_idx)]


pd.Series(labels).value_counts()

am    1995
af    1620
cm     744
cf     741
dtype: int64

## Very strong clf for race

*note, balanced didn't make much of a difference in performance of the model but it made a slight amount with the probabilities weights which is why I included it as it invovles less noise with the qualitative analysis and someone's likely to poke holes in the rigor of the analysis so why not.

In [6]:
rf = LogisticRegression(class_weight='balanced').fit(encodings[:-num_of_friends], labels)
get_multi_label_roc_score(pd.Series(labels), encodings[:-num_of_friends] ,rf)

0.9975900651259053

In [7]:
df = pd.DataFrame(rf.predict_proba(encodings)).applymap(lambda x: '%.3f' % x)
df.columns = ['cm','am', 'af', 'cf']

df.index = map(lambda x: x.split('/')[-1].replace('_face.jpg',''), paths)
df = df.astype(float)
df['rating'] = colors_dic
df['labels'] = labels + [np.nan for _ in range(df.__len__() - len(labels))]

# Asian females rated the most attractive overall

CFs close 2nd and AZN men last which makes me the question the rater's demographic more. Are they all CF men?

In [8]:
print (df[df['labels'] == 'cm']['rating'].mean(), df[df['labels'] == 'cm']['rating'].__len__())
print (df[df['labels'] == 'cf']['rating'].mean(), df[df['labels'] == 'cf']['rating'].__len__())
print (df[df['labels'] == 'am']['rating'].mean(), df[df['labels'] == 'am']['rating'].__len__())
print (df[df['labels'] == 'af']['rating'].mean(), df[df['labels'] == 'af']['rating'].__len__())

3.2594086021505375 744
3.406207827260459 741
3.0791979949874686 1995
3.4987654320987653 1620


## Moderate but reliable to attraction score prediction

Now it's fair to say there's a pattern! This shows in our Umaps as a continuum across the cluster from most attractive on one of hte cluster to least attractive.

In [9]:
rf = LogisticRegression(class_weight='balanced').fit(encodings[:-num_of_friends], colors_dic[:-num_of_friends])
get_multi_label_roc_score(pd.Series(colors_dic[:-num_of_friends]), encodings[:-num_of_friends] ,rf)

0.7665963430592047


```
### Adding attraction vector to the images, no difference in performance

So I accidently included the label in the feature matrix as a column and it made NO difference in the performance... this makes me question the classifier (LR) in it's ability... strange.

which means there's unlikely to be a racial bias within the ratings.
```

In [11]:
X = np.c_[encodings[:-num_of_friends], df['labels'][:-num_of_friends]]

rf = LogisticRegression(class_weight='balanced').fit(X, colors_dic[:-num_of_friends])
get_multi_label_roc_score(pd.Series(colors_dic[:-num_of_friends]), encodings[:-num_of_friends] ,rf)

ValueError: could not convert string to float: 'cf'

In [12]:
_, paths, encodings, arrays_rescaled, X = load_encodings('../data/tinder_pics_dislikes_faces_firsts_deduped_encodings.pickle')

paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/tinder_pics_likes_faces_deduped_firsts_encodings.pickle')
# paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/tinder_pics_2017_deduped_firsts_faces_encodings.pickle')

num_of_dislikes = len(_)
num_of_dislikes

743 743 743 743


638

In [13]:
tinder_labels = np.r_[np.zeros(num_of_dislikes), np.ones(len(paths) - num_of_dislikes )]
tinder_labels.shape

(743,)

In [15]:
rf = LogisticRegression(class_weight='balanced').fit(encodings, tinder_labels)
# (clf, feature_M, labels, class_imbalance=True)
score_classifier(rf, encodings, tinder_labels)

(0.7559084411323586,
 0.7574404761904762,
 0.04489589750987322,
 array([0.68229167, 0.74888393, 0.75744048, 0.82264717, 0.76827897]))

In [16]:
paths, encodings, arrays_rescaled, X = append_new_dataset(paths, encodings, arrays_rescaled, X, '../data/tinder_pics_2017_deduped_firsts_faces_encodings.pickle')
tinder_labels = np.r_[np.zeros(num_of_dislikes), np.ones(len(paths) - num_of_dislikes )]
tinder_labels.shape

994 994 994 994


(994,)

## Better with more images

with 2017 appended nearly 0.7914646779270329, so there's definitely a pattern, if I wanted to train it. further I could just add all pictures instead of just the firsts. 

How to figure out what explicitly is this pattern? This is also needed above for the attraction scores

methods to augment images:
------------
- blurr, 
- jitter, 
- random block, 
- picture mirror even orientation, 
- include all instead of first

In [21]:
rf = LogisticRegression(class_weight='balanced').fit(encodings, tinder_labels)

score_classifier(rf, encodings, tinder_labels)

(0.79185908628994,
 0.7889539930555555,
 0.026717800671208273,
 array([0.78895399, 0.80501761, 0.78697183, 0.7482533 , 0.8300987 ]))